<a href="https://colab.research.google.com/github/ryan-cqx/Coursera-Capstone-Project/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Segmenting and Clustering Neighborhoods in Toronto***

Import libraries

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [0]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

**Build pandas dataframe**

In [0]:
df_toronto = pd.DataFrame(data, columns=['Postcode', 'Borough', 'Neighborhood'])
df_toronto.columns = ['Postcode', 'Borough', 'Neighborhood']

**Data Processing**

In [0]:
df_toronto.drop(df_toronto[df_toronto['Borough'] == 'Not assigned'].index, inplace=True)
df_toronto.loc[df_toronto.Neighborhood == 'Not assigned', "Neighborhood"] = df_toronto.Borough


## Group data based on postcode and Borough

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [0]:
df_grp = df_toronto.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grp.columns = ['Postcode', 'Borough', 'Neighborhood']

In [74]:
df_grp.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [73]:
df_grp.shape

(103, 3)